## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

os.chdir("..")
print(os.path.abspath(os.curdir))

C:\Users\14074\Desktop\Bootcamp_info\World_Weather_Analysis


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vardo,NO,70.3705,31.1107,35.06,93,100,8.79,overcast clouds
1,1,Saldanha,ZA,-33.0117,17.9442,55.45,61,96,18.66,overcast clouds
2,2,Caravelas,BR,-17.7125,-39.2481,78.30,81,100,5.73,overcast clouds
3,3,Umzimvubu,BR,-17.7125,-39.2481,78.30,81,100,5.73,overcast clouds
4,4,Sawakin,SD,19.1059,37.3321,70.70,71,40,9.86,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp_req = int(input("What temperature would you not like your vacation to exceed in degrees F? "))
min_temp_req = int(input("What temperature would you not like your vacation to fall below in degrees F? "))

What temperature would you not like your vacation to exceed in degrees F? 90
What temperature would you not like your vacation to fall below in degrees F? 70


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[((city_data_df["Max Temp"] < max_temp_req) & (city_data_df["Max Temp"] > min_temp_req))]

In [6]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1985,False,False,False,False,False,False,False,False,False,False
1987,False,False,False,False,False,False,False,False,False,False
1991,False,False,False,False,False,False,False,False,False,False
1992,False,False,False,False,False,False,False,False,False,False


In [7]:
# 4b. Drop any duplicates and create a new DataFrame that doesn’t have duplicates. No empty rows present.
city_list = list(city_data_df["City"].drop_duplicates(False,False))
clean_df = city_data_df[city_data_df["City"].isin(city_list)]
clean_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Umzimvubu,BR,-17.7125,-39.2481,78.30,81,100,5.73,overcast clouds
4,4,Sawakin,SD,19.1059,37.3321,70.70,71,40,9.86,scattered clouds
5,5,Kulhudhuffushi,MV,6.6221,73.0700,81.68,73,97,4.61,overcast clouds
28,28,Beyla,GN,8.6833,-8.6333,77.04,54,72,5.03,broken clouds
39,39,Kibala,TD,9.1104,18.3463,87.89,28,24,4.14,few clouds
...,...,...,...,...,...,...,...,...,...,...
1961,1961,Manta,EC,-0.9500,-80.7333,82.18,78,40,4.61,scattered clouds
1970,1970,Koregaon,IN,18.6500,74.0833,74.19,22,2,5.70,clear sky
1972,1972,Zeerust,ZA,-25.5369,26.0751,70.86,50,81,3.15,broken clouds
1983,1983,Maunabo,PR,18.0072,-65.8993,78.64,79,14,12.08,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Umzimvubu,BR,78.30,overcast clouds,-17.7125,-39.2481,
4,Sawakin,SD,70.70,scattered clouds,19.1059,37.3321,
5,Kulhudhuffushi,MV,81.68,overcast clouds,6.6221,73.0700,
28,Beyla,GN,77.04,broken clouds,8.6833,-8.6333,
39,Kibala,TD,87.89,few clouds,9.1104,18.3463,
59,Yorosso,ML,85.66,overcast clouds,12.3536,-4.7761,
61,Lodja,CD,70.07,overcast clouds,-3.4833,23.4333,
68,Unguia,CO,80.10,scattered clouds,8.0434,-77.0945,
70,Savannah Bight,HN,80.92,scattered clouds,16.4500,-85.8500,
75,Ikom,NG,80.15,overcast clouds,5.9667,8.7063,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for i, c in enumerate(hotel_df["City"]):
    
    # 6c. Get latitude and longitude from DataFrame
    lat_val = float(hotel_df.loc[hotel_df["City"] == c]["Lat"])
    lng_val = float(hotel_df.loc[hotel_df["City"] == c]["Lng"])
    params["location"] = f"{lat_val},{lng_val}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("",np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Umzimvubu,BR,78.30,overcast clouds,-17.7125,-39.2481,DIAN SINKO
4,Sawakin,SD,70.70,scattered clouds,19.1059,37.3321,hotel safari
5,Kulhudhuffushi,MV,81.68,overcast clouds,6.6221,73.0700,Maison d'arrêt et de correction de Yoro
28,Beyla,GN,77.04,broken clouds,8.6833,-8.6333,castelo pizzaria
39,Kibala,TD,87.89,few clouds,9.1104,18.3463,Hotel le Carrefour
61,Lodja,CD,70.07,overcast clouds,-3.4833,23.4333,Tibati Hotel
68,Unguia,CO,80.10,scattered clouds,8.0434,-77.0945,Dormitório labelle
70,Savannah Bight,HN,80.92,scattered clouds,16.4500,-85.8500,Le Pilotis
75,Ikom,NG,80.15,overcast clouds,5.9667,8.7063,Almupaark complex
77,Kanniyakumari,IN,79.12,overcast clouds,8.0833,77.5667,Doo Palace Hotel


In [11]:
# 8a. Create the output File (CSV)
import csv
output_data_file = "Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd></dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))